In [80]:
import pandas as pd
import numpy as np
import plotly.express as px
import nbformat
import plotly.io as pio

In [81]:
# Set plotly theme to 'plotly_white'
pio.templates.default = 'plotly_white'

# Load data

TRAIN.CSV

- row_id: (int64) ID code for the row.
- timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.
- user_id: (int32) ID code for the user.
- content_id: (int16) ID code for the user interaction
- content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.
- task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.
- user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.
- answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.
- prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.
- prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

In [ ]:
# Define the data types for each column
dtype_train_dict = {
    'row_id': 'Int64',
    'timestamp': 'Int64',
    'user_id': 'Int32',
    'content_id': 'Int16',
    'content_type_id': 'Int8',
    'task_container_id': 'Int16',
    'user_answer': 'Int8',
    'answered_correctly': 'Int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

# Read the CSV file with specified dtypes
train = pd.read_csv('train.csv', dtype=dtype_train_dict)

In [29]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int8   
 1   timestamp                       int8   
 2   user_id                         int8   
 3   content_id                      int8   
 4   content_type_id                 int8   
 5   task_container_id               int8   
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int8(8), object(1)
memory usage: 4.9 GB


In [30]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,60,0,1,3,1,NaN,NaN
1,1,111,115,84,0,2,2,1,37000.0,False
2,2,91,115,-128,0,0,0,1,55000.0,False
3,3,95,115,-76,0,3,0,1,19000.0,False
4,4,-19,115,-14,0,4,1,1,11000.0,False


In [31]:
train = train.sample(frac=0.1, random_state=42)

QUESTIONS.CSV: METADATA FOR THE QUESTIONS POSED TO USERS.

- question_id: foreign key for the train/test content_id column, when the content type is question (0).
- bundle_id: code for which questions are served together.
- correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.
- part: the relevant section of the TOEIC test.
- tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [74]:
# Define the data types for each column
dtype_questions_dict = {
    'question_id': 'int16',
    'bundle_id': 'int16',
    'correct_answer': 'int8',
    'part': 'int16',
    'tags': 'int16'
}

# Read the CSV file with specified dtypes
questions = pd.read_csv('questions.csv', dtype=dtype_questions_dict)

In [75]:
questions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int16 
 1   bundle_id       13523 non-null  int16 
 2   correct_answer  13523 non-null  int8  
 3   part            13523 non-null  int16 
 4   tags            13522 non-null  object
dtypes: int16(3), int8(1), object(1)
memory usage: 826.0 KB


In [78]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


LECTURES.CSV: METADATA FOR THE LECTURES WATCHED BY USERS AS THEY PROGRESS IN THEIR EDUCATION.

- lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).
- part: top level category code for the lecture.
- tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.
- type_of: brief description of the core purpose of the lecture.

In [35]:
# Define the data types for each column
dtype_questions_dict = {
    'lecture_id': 'int16',
    'tag': 'int16',
    'part': 'int8'
}

# Read the CSV file with specified dtypes
lectures = pd.read_csv('lectures.csv', dtype=dtype_questions_dict)

In [36]:
lectures.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int8  
 1   tag         418 non-null    int8  
 2   part        418 non-null    int8  
 3   type_of     418 non-null    object
dtypes: int8(3), object(1)
memory usage: 25.9 KB


In [37]:
lectures.head()

,lecture_id,tag,part,type_of
0,89,-97,5,concept
1,100,70,1,concept
2,-71,45,6,concept
3,-64,79,5,solving question
4,61,-100,5,solving question


# Check train

In [38]:
# Check for missing values in each column
missing_values = train.isna().sum()
print(missing_values)

row_id                                 0
timestamp                              0
user_id                                0
content_id                             0
content_type_id                        0
task_container_id                      0
user_answer                            0
answered_correctly                     0
prior_question_elapsed_time       235186
prior_question_had_explanation     39329
dtype: int64


In [ ]:
train.drop(["row_id"], axis=1)

In [39]:
# Check for duplicates based on all columns
duplicates = train.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 1143


In [40]:
# Count the number of rows before dropping duplicates
num_rows_before = train.shape[0]
print(f"Number of rows before dropping duplicates: {num_rows_before}")

# Drop duplicates (keeping the first occurrence by default)
train.drop_duplicates(subset=None, keep='first', inplace=True)

# Count the number of rows after dropping duplicates
num_rows_after = train.shape[0]
print(f"Number of rows after dropping duplicates: {num_rows_after}")

# Print the difference
num_duplicates_dropped = num_rows_before - num_rows_after
print(f"Number of duplicates dropped: {num_duplicates_dropped}")

Number of rows before dropping duplicates: 10123033
Number of rows after dropping duplicates: 10121890
Number of duplicates dropped: 1143


In [41]:
# convert 'prior_question_elapsed_time' to seconds
train['prior_question_elapsed_time'] = train['prior_question_elapsed_time'] / 1000

In [42]:
# Groupby 'user_answer' and count the occurrences
user_answer_counts = train['user_answer'].value_counts().reset_index()
user_answer_counts.columns = ['user_answer', 'count']

# Plot the counts of 'user_answer' using Plotly
fig = px.bar(user_answer_counts, x='user_answer', y='count', 
             title='Distribution of User Answers',
             labels={'user_answer': 'User Answer', 'count': 'Count'})
fig.show()

In [43]:
# Groupby 'answered_correctly' and count the occurrences
answered_correctly_counts = train['answered_correctly'].value_counts().reset_index()
answered_correctly_counts.columns = ['answered_correctly', 'count']

# Plot the counts of 'answered_correctly' using Plotly
fig = px.bar(answered_correctly_counts, x='answered_correctly', y='count',
             title='Distribution of Answers (Correct/Incorrect)',
             labels={'answered_correctly': 'Answered Correctly', 'count': 'Count'})
fig.show()

In [44]:
# check percentage of unknown 'answered_correctly'
percentage = (train['answered_correctly'] == -1).mean() * 100
print(f"Percentage of rows with unknown answered: {percentage:.2f}%")

Percentage of rows with unknown answered: 1.93%


In [45]:
# Separate content into questions and lectures
questions_train_df = train[train['content_type_id'] == 0]
lectures_train_df = train[train['content_type_id'] == 1]

In [ ]:
questions_train_df = pd.merge(
    left=questions_train_df, right=questions, 
    left_on="content_id", right_on="question_id",
)

questions_train_df = questions_train_df.drop(["question_id"], axis=1)

In [ ]:
lectures_train_df = pd.merge(
    left=lectures_train_df, right=lectures, 
    left_on="content_id", right_on="lecture_id",
)

lectures_train_df = lectures_train_df.drop(["lecture_id"], axis=1)

Analysis 1: Impact of Prior Explanation on Correct Answer Rate

In [20]:
explanation_group = questions_train_df.groupby('prior_question_had_explanation')['answered_correctly'].mean().reset_index()
lectures_train_merged = lectures_train_df.merge(lectures, left_on='content_id', right_on='lecture_id', how='left')

fig1 = px.bar(
    explanation_group,
    x='prior_question_had_explanation',
    y='answered_correctly',
    title='Impact of Prior Explanation on Correct Answer Rate',
    labels={'prior_question_had_explanation': 'Prior Question Had Explanation', 'answered_correctly': 'Average Correct Answer Rate'},
    text='answered_correctly'
)
fig1.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig1.show()

In [21]:
explanation_correct = questions_train_df.groupby(['prior_question_had_explanation', 'answered_correctly']).size().reset_index(name='count')
fig = px.bar(explanation_correct, x='prior_question_had_explanation', y='count', color='answered_correctly',
             title='Impact of Prior Question Explanation on Answer Correctness',
             labels={'prior_question_had_explanation': 'Prior Question Had Explanation', 'count': 'Number of Responses', 'answered_correctly': 'Answered Correctly'})
fig.show()

Analysis 2: Average Time Taken vs Correct Answer Rate

In [22]:
time_correct_group = questions_train_df.groupby(pd.cut(questions_train_df['prior_question_elapsed_time'], bins=10))['answered_correctly'].mean().reset_index()
time_correct_group['prior_question_elapsed_time'] = time_correct_group['prior_question_elapsed_time'].astype(str)

fig2 = px.line(
    time_correct_group,
    x='prior_question_elapsed_time',
    y='answered_correctly',
    title='Average Time Taken vs Correct Answer Rate',
    labels={'prior_question_elapsed_time': 'Prior Question Elapsed Time (binned)', 'answered_correctly': 'Average Correct Answer Rate'}
)
fig2.show()

/var/folders/tn/pky4_jf56t74wlclcq15d9sc0000gn/T/ipykernel_13827/4160572691.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Analysis 3: Correct Answer Rate by Lecture Part

In [23]:
lectures_train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
89,89,-62,-70,-104,1,14,-1,-1,NaN,False
117,117,-89,6,96,1,21,-1,-1,NaN,False
212,-44,-75,6,-1,1,104,-1,-1,NaN,False
216,-40,17,6,113,1,121,-1,-1,NaN,False
295,39,37,-81,44,1,59,-1,-1,NaN,False


In [24]:
lectures_train_merged.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,lecture_id,tag,part,type_of
0,89,-62,-70,-104,1,14,-1,-1,NaN,False,-104,-127,2,intention
1,89,-62,-70,-104,1,14,-1,-1,NaN,False,-104,72,6,concept
2,89,-62,-70,-104,1,14,-1,-1,NaN,False,-104,111,1,concept
3,117,-89,6,96,1,21,-1,-1,NaN,False,96,53,6,solving question
4,117,-89,6,96,1,21,-1,-1,NaN,False,96,62,2,solving question


In [25]:
part_lecture_count = lectures_train_merged['part'].value_counts().reset_index()
part_lecture_count.columns = ['part', 'lecture_count']

fig3 = px.bar(
    part_lecture_count,
    x='part',
    y='lecture_count',
    title='Lecture Count by Part',
    labels={'part': 'Part', 'lecture_count': 'Lecture Count'},
    text='lecture_count'
)
fig3.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig3.show()


Analysis 4: Distribution of User Answers

In [26]:
user_answer_dist = questions_train_df['user_answer'].value_counts().reset_index()
user_answer_dist.columns = ['user_answer', 'count']

fig4 = px.bar(
    user_answer_dist,
    x='user_answer',
    y='count',
    title='Distribution of User Answers',
    labels={'user_answer': 'User Answer', 'count': 'Count'},
    text='count'
)
fig4.update_traces(texttemplate='%{text}', textposition='outside')
fig4.show()

Analysis 5: Average Correct Answer Rate by Task Container

In [27]:
task_container_group = questions_train_df[questions_train_df['answered_correctly'] != -1].groupby('task_container_id')['answered_correctly'].mean().reset_index()

fig5 = px.line(
    task_container_group,
    x='task_container_id',
    y='answered_correctly',
    title='Average Correct Answer Rate by Task Container',
    labels={'task_container_id': 'Task Container ID', 'answered_correctly': 'Average Correct Answer Rate'}
)
fig5.show()

Analysis 6: Lecture Types Distribution

In [28]:
lecture_type_dist = lectures_train_merged['type_of'].value_counts().reset_index()
lecture_type_dist.columns = ['type_of', 'count']

fig6 = px.pie(
    lecture_type_dist,
    names='type_of',
    values='count',
    title='Lecture Types Distribution'
)
fig6.show()

Analysis 7: Lecture Types

In [29]:
lecture_type_correct = lectures_train_merged.groupby('type_of').agg(
    lecture_count=('type_of', 'size'),
    avg_correct_answer_rate=('answered_correctly', 'mean')
).reset_index()
lecture_type_correct.dropna(subset=['avg_correct_answer_rate'], inplace=True)

fig7 = px.bar(
    lecture_type_correct,
    x='type_of',
    y='lecture_count',
    title='Lecture Count by Type',
    labels={'type_of': 'Lecture Type', 'lecture_count': 'Lecture Count'},
    text='lecture_count'
)
fig7.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig7.show()

Analysis 8: Correct Answer Rate by User ID

In [30]:
user_correct_group = train[train['answered_correctly'] != -1].groupby('user_id')['answered_correctly'].mean().reset_index()
fig = px.histogram(user_correct_group, x='answered_correctly', nbins=30,
                   title='Distribution of Average Correct Answer Rate by User',
                   labels={'answered_correctly': 'Average Correct Answer Rate', 'count': 'Number of Users'})
fig.show()

Analysis 8: Average Elapsed Time by Correct/Incorrect Answers

In [31]:
# fig = px.box(questions_train_df, x='answered_correctly', y='prior_question_elapsed_time',
#              title='Prior Question Elapsed Time by Answer Correctness',
#              labels={'answered_correctly': 'Answered Correctly', 'prior_question_elapsed_time': 'Elapsed Time (milliseconds)'})
# fig.show()

: 

In [34]:
# concatenated_df = pd.concat([questions_train_df, lectures_train_df], axis=0, ignore_index=True)

In [35]:
# lectures_train_df.head()

In [36]:
# questions_train_df.head()

In [37]:
# concatenated_df.head()

In [38]:
# # Import necessary libraries
# import pandas as pd
# import plotly.express as px

# # Assuming `questions_train_merged` and `lectures_train_df` are already loaded DataFrames

# # Get unique user_ids from both datasets
# user_ids_lectures = set(lectures_train_df['user_id'].unique())
# user_ids_questions = set(questions_train_df['user_id'].unique())

# # Check which users attended lectures and answered questions
# users_in_lectures = user_ids_questions.intersection(user_ids_lectures)
# users_not_in_lectures = user_ids_questions - user_ids_lectures

# # Filter records of users who attended lectures and those who did not
# users_in_lectures_df = questions_train_df[questions_train_df['user_id'].isin(users_in_lectures)]
# users_not_in_lectures_df = questions_train_df[questions_train_df['user_id'].isin(users_not_in_lectures)]

# # Calculate the average correct answer rate for users who attended lectures and those who did not
# user_correct_answer_rate_in_lectures = users_in_lectures_df.groupby('user_id')['answered_correctly'].mean().reset_index()
# user_correct_answer_rate_in_lectures['group'] = 'Attended Lectures'

# user_correct_answer_rate_not_in_lectures = users_not_in_lectures_df.groupby('user_id')['answered_correctly'].mean().reset_index()
# user_correct_answer_rate_not_in_lectures['group'] = 'Did Not Attend Lectures'

# # Concatenate the two groups
# user_correct_answer_rate = pd.concat([user_correct_answer_rate_in_lectures, user_correct_answer_rate_not_in_lectures], ignore_index=True)

# # Plot the average correct answer rate for users attending vs not attending lectures using Plotly
# fig = px.histogram(user_correct_answer_rate, x='answered_correctly', color='group', barmode='overlay', nbins=30,
#                    title='Distribution of Average Correct Answer Rate: Attended vs Did Not Attend Lectures',
#                    labels={'answered_correctly': 'Average Correct Answer Rate', 'count': 'Number of Users', 'group': 'User Group'},
#                    color_discrete_sequence=['skyblue', 'salmon'])
# fig.update_layout(xaxis_title='Average Correct Answer Rate', yaxis_title='Number of Users')

# # Show the plot
# fig.show()